# 1 模型构造
让我们回顾一下在3.10节（多层感知机的简洁实现）中含单隐藏层的多层感知机的实现方法。我们首先构造Sequential实例，然后依次添加两个全连接层。其中第一层的输出大小为256，即隐藏层单元个数是256；第二层的输出大小为10，即输出层单元个数是10。我们在上一章的其他节中也使用了Sequential类构造模型。这里我们介绍另外一种基于Module类的模型构造方法：它让模型构造更加灵活。

## 1.1 继承Module类来构造模型
Module类是nn模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。下面继承Module类构造本节开头提到的多层感知机。这里定义的MLP类重载了Module类的__init__函数和forward函数。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

In [3]:
import torch
from torch import nn

class MLP(nn.Module):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10) # 输出层
        
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

mlp =MLP()
print(mlp)

        

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


以上的MLP类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的backward函数。

我们可以实例化MLP类得到模型变量net。下面的代码初始化net并传入输入数据X做一次前向计算。其中，net(X)会调用MLP继承自Module类的__call__函数，这个函数将调用MLP类定义的forward函数来完成前向计算。

In [25]:
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)


MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.1905, -0.0697, -0.0802, -0.1243,  0.0072,  0.0479, -0.0081, -0.0120,
         -0.1664, -0.1323],
        [-0.0108, -0.1737, -0.1647, -0.0688, -0.0262,  0.1896,  0.0235,  0.0867,
         -0.1466, -0.2733]], grad_fn=<AddmmBackward0>)

注意，这里并没有将Module类命名为Layer（层）或者Model（模型）之类的名字，这是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如PyTorch提供的Linear类），又可以是一个模型（如这里定义的MLP类），或者是模型的一个部分。我们下面通过两个例子来展示它的灵活性。

## 1.2 Module的子类
我们刚刚提到，Module类是一个通用的部件。事实上，PyTorch还实现了继承自Module的可以方便构建模型的类: 如Sequential、ModuleList和ModuleDict等等。

### 1.2.1 Sequential类
当模型的前向计算为简单串联各个层的计算时，Sequential类可以通过更加简单的方式定义模型。这正是Sequential类的目的：它可以接收一个子模块的有序字典（OrderedDict）或者一系列子模块作为参数来逐一添加Module的实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

下面我们实现一个与Sequential类有相同功能的MySequential类。这或许可以帮助读者更加清晰地理解Sequential类的工作机制。

In [27]:
from collections import OrderedDict
class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict):  # 如果传入的是一个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)    # add_module方法会将module添加进self._modules(一个OrderedDict)
        else:   # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    
    def forward(self, input):
        # self._modules返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成员
        for module in self._modules.values():
            print(module)
            input = module(input)
        return input

我们用MySequential类来实现前面描述的MLP类，并使用随机初始化的模型做一次前向计算。

In [28]:
net = MySequential(
        OrderedDict({'linear1': nn.Linear(784, 256),
        'activa': nn.ReLU(),
        'linear2': nn.Linear(256, 10), 
        }))
print(net)
print(net(X))


MySequential(
  (linear1): Linear(in_features=784, out_features=256, bias=True)
  (activa): ReLU()
  (linear2): Linear(in_features=256, out_features=10, bias=True)
)
Linear(in_features=784, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)
tensor([[ 0.0035,  0.0867,  0.1514, -0.0153, -0.1171,  0.0960,  0.2596,  0.2833,
         -0.0320,  0.1086],
        [ 0.0201, -0.0538,  0.2013, -0.0115, -0.2148,  0.1876,  0.1363,  0.2533,
          0.0308,  0.1522]], grad_fn=<AddmmBackward0>)


### 1.2.2 ModuleList类
ModuleList接收一个子模块的列表作为输入，然后也可以类似List那样进行append和extend操作:

In [31]:
net = nn.ModuleList([nn.Linear(784,256), nn.ReLU()])
net.append(nn.Linear(256, 10))  # 类似List的append操作
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


既然Sequential和ModuleList都可以进行列表化构造网络，那二者区别是什么呢。ModuleList仅仅是一个储存各种模块的列表，这些模块之间没有联系也没有顺序（所以不用保证相邻层的输入输出维度匹配），而且没有实现forward功能需要自己实现，所以上面执行net(torch.zeros(1, 784))会报NotImplementedError；而Sequential内的模块需要按照顺序排列，要保证相邻层的输入输出大小相匹配，内部forward功能已经实现。

ModuleList的出现只是让网络定义前向传播时更加灵活，见下面官网的例子。

In [33]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(10)])

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x

net = MyModule()
print(net)


MyModule(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=10, bias=True)
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): Linear(in_features=10, out_features=10, bias=True)
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): Linear(in_features=10, out_features=10, bias=True)
  )
)


另外，ModuleList不同于一般的Python的list，加入到ModuleList里面的所有模块的参数会被自动添加到整个网络中，下面看一个例子对比一下。

In [36]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10)])

class Module_List(nn.Module):
    def __init__(self):
        super(Module_List, self).__init__()
        self.linears = [nn.Linear(10, 10)]

net1 = Module_ModuleList()
net2 = Module_List()

print("net1:")
for p in net1.named_parameters():
    print(p)

print("net2:")
for p in net2.parameters():
    print(p)


net1:
('linears.0.weight', Parameter containing:
tensor([[-0.1508,  0.2951,  0.1897, -0.2365, -0.2448, -0.3085, -0.2723,  0.2022,
          0.0040, -0.1177],
        [-0.0011, -0.2683,  0.1903, -0.1655,  0.1287, -0.1671, -0.2551,  0.0912,
         -0.2226, -0.1715],
        [ 0.1222,  0.1007, -0.1130,  0.1796, -0.1038, -0.2267,  0.0850, -0.3126,
         -0.2573, -0.1444],
        [-0.0934, -0.0593, -0.0775, -0.3144, -0.0529, -0.0409,  0.3125,  0.0193,
          0.3013,  0.2155],
        [-0.0472,  0.2361, -0.0144, -0.2260,  0.0046, -0.0944,  0.1793, -0.0771,
          0.0941,  0.1933],
        [ 0.2428,  0.1823, -0.2349,  0.1263, -0.2446, -0.0612, -0.0664,  0.0621,
          0.1086, -0.2201],
        [ 0.0063, -0.2365, -0.1902, -0.0857, -0.1980,  0.2177, -0.1914, -0.1145,
          0.1049,  0.1317],
        [-0.0927, -0.2122,  0.2509, -0.2851, -0.1617, -0.0648,  0.2784, -0.1244,
          0.0119, -0.1440],
        [ 0.0628,  0.1513,  0.1950, -0.0104,  0.1811,  0.2000,  0.2656,  0.1353

### 1.2.3 ModuleDict类
ModuleDict接收一个子模块的字典作为输入, 然后也可以类似字典那样进行添加访问操作:

In [38]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)
# net(torch.zeros(1, 784)) # 会报NotImplementedError



Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


和ModuleList一样，ModuleDict实例仅仅是存放了一些模块的字典，并没有定义forward函数需要自己定义。同样，ModuleDict也与Python的Dict有所不同，ModuleDict里的所有模块的参数会被自动添加到整个网络中。

## 1.3 构造复杂的模型
虽然上面介绍的这些类可以使模型构造更加简单，且不需要定义forward函数，但直接继承Module类可以极大地拓展模型构造的灵活性。下面我们构造一个稍微复杂点的网络FancyMLP。在这个网络中，我们通过get_constant函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用Tensor的函数和Python的控制流，并多次调用相同的层。

In [39]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()


在这个FancyMLP模型中，我们使用了常数权重rand_weight（注意它不是可训练模型参数）、做了矩阵乘法操作（torch.mm）并重复使用了相同的Linear层。下面我们来测试该模型的前向计算。

In [40]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)


FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-9.5110, grad_fn=<SumBackward0>)

因为FancyMLP和Sequential类都是Module类的子类，所以我们可以嵌套调用它们。

In [41]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU()) 

    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)


Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-8.4967, grad_fn=<SumBackward0>)

# 小结
* 可以通过继承Module类来构造模型。
* Sequential、ModuleList、ModuleDict类都继承自Module类。
* 与Sequential不同，ModuleList和ModuleDict并没有定义一个完整的网络，它们只是将不同的模块存放在一起，需要自己定义forward函数。
* 虽然Sequential等类可以使模型构造更加简单，但直接继承Module类可以极大地拓展模型构造的灵活性。
